In [2]:
import networkx as nx

import matplotlib.pyplot as plt

# Create a 3D grid graph
G = nx.grid_graph(dim=[8, 8, 8])



In [27]:
import numpy as np

import plotly.graph_objects as go

# Define the number of points in each dimension
num_points = 8

# Generate evenly spaced coordinates
x_coords = np.linspace(-1, 1, num_points)
y_coords = np.linspace(-1, 1, num_points)
z_coords = np.linspace(-1, 1, num_points)

# Create a meshgrid of coordinates
X, Y, Z = np.meshgrid(x_coords, y_coords, z_coords)

# Reshape the coordinates into a 1D array
x_coords = X.flatten()
y_coords = Y.flatten()
z_coords = Z.flatten()

# Create a dictionary to store the coordinates
pos = {}
for i in range(len(x_coords)):
    pos[i] = np.array([x_coords[i], y_coords[i], z_coords[i]])

# Create a scatter3d trace
trace = go.Scatter3d(
    x=[node[0] for node in G.nodes()],
    y=[node[1] for node in G.nodes()],
    z=[node[2] for node in G.nodes()],
    mode='markers',
    marker=dict(
        size=5,
        color='blue'
    )
)

# Create the layout
layout = go.Layout(
    title='3D Graph',
    scene=dict(
        xaxis=dict(title='X'),
        yaxis=dict(title='Y'),
        zaxis=dict(title='Z')
    )
)


# Create the figure
fig = go.Figure(data=[trace], layout=layout)


# Create a list to store the edge traces
edge_traces = []

# Iterate over each edge in the graph
for edge in G.edges:
    # Get the start and end coordinates of the edge
    start = edge[0]
    end = edge[1]
    
    # Create a scatter3d trace for the edge
    edge_trace = go.Scatter3d(
        x=[start[0], end[0]],
        y=[start[1], end[1]],
        z=[start[2], end[2]],
        mode='lines',
        line=dict(color='red', width=2)
    )
    
    # Add the edge trace to the list
    edge_traces.append(edge_trace)

# Add the edge traces to the figure
fig.add_traces(edge_traces)

# Show the figure
fig.show()

In [13]:
# Create a mapping from node coordinates to unique integer IDs
coord_to_id = {coord: i for i, coord in enumerate(G.nodes())}
id_to_coord = {i: coord for coord, i in coord_to_id.items()}



{(0, 0, 0): 0,
 (0, 0, 1): 1,
 (0, 0, 2): 2,
 (0, 0, 3): 3,
 (0, 0, 4): 4,
 (0, 0, 5): 5,
 (0, 0, 6): 6,
 (0, 0, 7): 7,
 (0, 1, 0): 8,
 (0, 1, 1): 9,
 (0, 1, 2): 10,
 (0, 1, 3): 11,
 (0, 1, 4): 12,
 (0, 1, 5): 13,
 (0, 1, 6): 14,
 (0, 1, 7): 15,
 (0, 2, 0): 16,
 (0, 2, 1): 17,
 (0, 2, 2): 18,
 (0, 2, 3): 19,
 (0, 2, 4): 20,
 (0, 2, 5): 21,
 (0, 2, 6): 22,
 (0, 2, 7): 23,
 (0, 3, 0): 24,
 (0, 3, 1): 25,
 (0, 3, 2): 26,
 (0, 3, 3): 27,
 (0, 3, 4): 28,
 (0, 3, 5): 29,
 (0, 3, 6): 30,
 (0, 3, 7): 31,
 (0, 4, 0): 32,
 (0, 4, 1): 33,
 (0, 4, 2): 34,
 (0, 4, 3): 35,
 (0, 4, 4): 36,
 (0, 4, 5): 37,
 (0, 4, 6): 38,
 (0, 4, 7): 39,
 (0, 5, 0): 40,
 (0, 5, 1): 41,
 (0, 5, 2): 42,
 (0, 5, 3): 43,
 (0, 5, 4): 44,
 (0, 5, 5): 45,
 (0, 5, 6): 46,
 (0, 5, 7): 47,
 (0, 6, 0): 48,
 (0, 6, 1): 49,
 (0, 6, 2): 50,
 (0, 6, 3): 51,
 (0, 6, 4): 52,
 (0, 6, 5): 53,
 (0, 6, 6): 54,
 (0, 6, 7): 55,
 (0, 7, 0): 56,
 (0, 7, 1): 57,
 (0, 7, 2): 58,
 (0, 7, 3): 59,
 (0, 7, 4): 60,
 (0, 7, 5): 61,
 (0, 7, 6): 62,
 (

In [28]:
subgrid_size = (2, 2)
subgrid_to_nodes = {}
node_to_subgrid = {}

for node in G.nodes():
    x, y, z = node
    subgrid_x = x // subgrid_size[0]
    subgrid_y = y // subgrid_size[1]
    subgrid = (subgrid_x, subgrid_y)
    
    if subgrid not in subgrid_to_nodes:
        subgrid_to_nodes[subgrid] = []
    
    subgrid_to_nodes[subgrid].append(node)
    node_to_subgrid[node] = subgrid

In [29]:
import numpy as np

import plotly.graph_objects as go

# create color map for subgrids
colors = [i*50 for i in range(len(subgrid_to_nodes.keys()))]
# dict to store subgrid to color mapping
subgrid_to_color = {}
# iterate over subgrids and assign a color
for i, subgrid in enumerate(subgrid_to_nodes.keys()):
    subgrid_to_color[subgrid] = colors[i]
# Create a scatter3d trace with color based on subgrid

# choose a color palette with random colors

colorpalette = [[i, 'rgb({},{},{})'.format(np.random.randint(0, 255), np.random.randint(0, 255), np.random.randint(0, 255))] for i in np.linspace(0, 1, len(subgrid_to_nodes.keys()))]
trace = go.Scatter3d(
    x=[node[0] for node in G.nodes()],
    y=[node[1] for node in G.nodes()],
    z=[node[2] for node in G.nodes()],
    mode='markers',
    marker=dict(
        size=5,
        color=[subgrid_to_color[node_to_subgrid[node]] for node in G.nodes()],
        colorscale=colorpalette,
        colorbar=dict(title='Subgrid')
    )
)

# Create the layout
layout = go.Layout(
    title='3D Graph',
    scene=dict(
        xaxis=dict(title='X'),
        yaxis=dict(title='Y'),
        zaxis=dict(title='Z')
    )
)

# Create the figure
fig = go.Figure(data=[trace], layout=layout)

# Show the figure
fig.show()

In [5]:
!pip install hydra-core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 2.1 MB/s eta 0:00:00 MB/s eta 0:00:0101
